In the following notebook, we investigate the use of SAS (subsets that maximize expected augmentation similarity) to select representative subsets that improve self-supervised learning in a text categorization task. We also investigate the use of multilingual embeddings to further strengthen the SAS selection.

# Setup

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U
!pip install nlpaug
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.6 MB/s eta 0:00:00


In [ ]:
import csv
from datasets import load_dataset, DatasetDict, concatenate_datasets, Dataset
import gensim.downloader as api
import nlpaug.augmenter.word as naw
import nltk
from nltk.corpus import wordnet
import numpy as np
import pandas as pd
import random
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments
from tqdm.auto import tqdm
# Delete some data from RAM to free up space for later processes
import gc

# Ensure you have the necessary NLTK data
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Import Google Drive for locally saved files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Global variables

In [ ]:
# Max length of text embedding tensors
global_seed = 0
max_length = 256
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set seed for reproducibility
np.random.seed(global_seed)
random.seed(global_seed)

## Helper functions

In [ ]:
# Basic preprocessing: lowercasing and removing non-alphanumeric characters
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    return text

## Load dataset

In [ ]:
# Load the AG News dataset
dataset = load_dataset("ag_news")

# The dataset is divided into 'train' and 'test' splits
train_data = dataset['train']
test_data = dataset['test']

# For the parts that use GPT, we will use only 2.5% of the data due to the cost of running GPT.
train_data = train_data.train_test_split(test_size=0.975, stratify_by_column="label", seed = global_seed)
train_data = train_data['train']

# Example: Viewing the first training sample
print(train_data[0])
print(len(train_data))

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

{'text': '9 hurt in blast at Indonesian Embassy PARIS -- An explosion struck the Indonesian Embassy in Paris today, slightly injuring nine people, a French radio station reported.', 'label': 0}
3000


## Create NLPAug augmented texts

In [ ]:
# aug_1 = naw.WordEmbsAug(
#     model_type='word2vec',
#     model_path='/content/drive/MyDrive/COM SCI 260D/word2vec-google-news-300.bin',
#     top_k = 5, # Only use top 5 options
#     aug_p = 0.2 # 20% of the words augmented
# )
#
# aug_2 = naw.WordEmbsAug(
#     model_type='word2vec',
#     model_path='/content/drive/MyDrive/COM SCI 260D/word2vec-google-news-300.bin',
#     top_k = 10, # Only use top 10 options
#     aug_p = 0.4 # 40% of the words augmented
# )
#
# def eda_augmentation_with_word2vec(sentence, type):
#     if type == 1:
#         return aug_1.augment(sentence)[0]
#     else:
#         return aug_2.augment(sentence)[0]

In [ ]:
# Add augmented texts for each example using NLPAug
# def add_nlp_aug_columns(data):
#     data['augment_1'] = eda_augmentation_with_word2vec(data['text'], 1)
#     data['augment_2'] = eda_augmentation_with_word2vec(data['text'], 2)
#     return data

# train_data = train_data.map(
#     lambda example: add_nlp_aug_columns(example)
# )

In [ ]:
# Save the augmentations as the process takes 2 hours
# train_df = train_data.to_pandas()
# train_df.to_json("/content/drive/MyDrive/COM SCI 260D/augmented_dataset.json")

In [ ]:
import datasets

# Load dataset with augmentations (pre-prepared)
train_df = pd.read_json("/content/drive/MyDrive/COM SCI 260D/augmented_dataset.json")
augmented_data = datasets.Dataset.from_pandas(train_df)

In [ ]:
print(augmented_data)
print(augmented_data['text'][-1])
print(augmented_data['text'][-1])

Dataset({
    features: ['text', 'label', 'augment_1', 'augment_2', '__index_level_0__'],
    num_rows: 3000
})
Palm Introduces a Memory-Packed Organizer (AP) AP - Handheld computer maker PalmOne Inc. unveiled Monday a new memory-packed personal digital assistant that can double as a portable data storage drive.
Palm Introduces a Memory-Packed Organizer (AP) AP - Handheld computer maker PalmOne Inc. unveiled Monday a new memory-packed personal digital assistant that can double as a portable data storage drive.


In [ ]:
# Merge datasets by columns
def add_augmented_columns(data, augments, idx):
    data['augment_1'] = augments['augment_1'][idx]
    data['augment_2'] = augments['augment_2'][idx]
    return data

train_data = train_data.map(
    lambda example, idx: add_augmented_columns(example, augmented_data, idx),
    with_indices=True
)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

## Load GPT augmented text and Spanish translations

In [ ]:
gpt_dataset = load_dataset("csv", data_files="/content/drive/MyDrive/COM SCI 260D/gpt_dataset.csv", split = 'train')

print(gpt_dataset)
print(gpt_dataset[0:5])
print(train_data[0:5])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['Original Text', 'Translation', 'Paraphrase 1', 'Paraphrase 2'],
    num_rows: 3000
})
{'Original Text': ['9 hurt in blast at Indonesian Embassy PARIS -- An explosion struck the Indonesian Embassy in Paris today, slightly injuring nine people, a French radio station reported.', "Indonesia Pursues Two Terror Suspects (AP) AP - Indonesian police pursued a manhunt Sunday for two of Southeast Asia's most wanted terrorism suspects while warning of the danger of more suicide attacks on Western interests in the wake of a lethal bombing outside the Australian Embassy.", 'Iran rejects call to stop atomic research Iran has rejected a UN resolution calling on the country to freeze uranium enrichment activities. It has also threatened to stop snap checks of its atomic facilities if Iran #39;s case is referred to the UN Security Council.', 'Hernndez #39;s Shoulder Worries Yanks he Yankees #39; game against the Minnesota Twins was rained out last night, but Orlando Hernndez,

In [ ]:
# Ensure both datasets have the same number of rows
assert len(gpt_dataset) == len(train_data), "Datasets must have the same number of rows"

# Merge datasets by columns
def add_gpt_columns(data, augments, idx):
    data['translation'] = augments['Translation'][idx]
    data['gpt_1'] = augments['Paraphrase 1'][idx]
    existing_augment = augments['Paraphrase 2'][idx]
    if not existing_augment or not existing_augment.strip():
        data['gpt_2'] = data['augment_1'] # Fallback to EDA augmentation
    else:
        data['gpt_2'] = existing_augment
    return data

train_data = train_data.map(
    lambda example, idx: add_gpt_columns(example, gpt_dataset, idx),
    with_indices=True
)

print(train_data)
print(train_data[0])

# There are 8 examples that are not correctly being imported. For now, we just remove them.
error_data = train_data.filter(lambda row: row['gpt_2'] is None or row['gpt_1'] is None)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'augment_1', 'augment_2', 'translation', 'gpt_1', 'gpt_2'],
    num_rows: 3000
})
{'text': '9 hurt in blast at Indonesian Embassy PARIS -- An explosion struck the Indonesian Embassy in Paris today, slightly injuring nine people, a French radio station reported.', 'label': 0, 'augment_1': '9 hurt in blast at Indonesian Embassy PARIS - - An thunderous_explosion hit the Indonesian Embassy in Pantheon_Sorbonne today, slightly injure three people, a Paris radio station reports.', 'augment_2': "2 hurt in explosion Tuesdayat Indonesian Embassy LYON_France - - An explosion struck in Indonesian charge_d'_affaires in Paris today, tad wounding nine people, a Algerian radio Finsbury_Park_Tube reported.", 'translation': '9 heridos en explosión en la Embajada de Indonesia PARÍS -- Una explosión golpeó la Embajada de Indonesia en París hoy, hiriendo levemente a nueve personas, según informó una estación de radio francesa.', 'gpt_1': '1. The Indonesian Embassy

Filter:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
print(error_data)

Dataset({
    features: ['text', 'label', 'augment_1', 'augment_2', 'translation', 'gpt_1', 'gpt_2'],
    num_rows: 0
})


In [ ]:
print(train_data['text'][0])
print(train_data['gpt_1'][0])
print(train_data['gpt_2'][0])
print(train_data['augment_1'][0])
print(train_data['augment_2'][0])
print("")
print(train_data['text'][-1])
print(train_data['gpt_1'][-1])
print(train_data['gpt_2'][-1])
print(train_data['augment_1'][-1])
print(train_data['augment_2'][-1])

9 hurt in blast at Indonesian Embassy PARIS -- An explosion struck the Indonesian Embassy in Paris today, slightly injuring nine people, a French radio station reported.
1. The Indonesian Embassy in Paris was struck by an explosion today, leading to minor injuries for nine individuals, as reported by a French radio station.
9 hurt in blast at Indonesian Embassy PARIS - - An thunderous_explosion hit the Indonesian Embassy in Pantheon_Sorbonne today, slightly injure three people, a Paris radio station reports.
9 hurt in blast at Indonesian Embassy PARIS - - An thunderous_explosion hit the Indonesian Embassy in Pantheon_Sorbonne today, slightly injure three people, a Paris radio station reports.
2 hurt in explosion Tuesdayat Indonesian Embassy LYON_France - - An explosion struck in Indonesian charge_d'_affaires in Paris today, tad wounding nine people, a Algerian radio Finsbury_Park_Tube reported.

Palm Introduces a Memory-Packed Organizer (AP) AP - Handheld computer maker PalmOne Inc. un

# Latent class discovery

First, we use 1% of the data randomly selected as a small piece of labelled information to train a basic pre-trained classifier and assign latent classes to each training data point.

In [ ]:
# Select 1% of the data randomly
shuffled_dataset = train_data.shuffle(seed = global_seed)
sample_size = int(0.01 * len(shuffled_dataset))
labeled_data = shuffled_dataset.select(range(sample_size))
rest_of_data = shuffled_dataset.select(range(sample_size, len(shuffled_dataset)))

# Get number of classes from the 1% data
unique_labels = set(labeled_data['label'])
num_classes = len(unique_labels)

print(num_classes)

4


In [ ]:
# Split the 1% dataset into training and validation sets. We only use 20% as validation set due to size of data.
labeled_data_for_training = labeled_data.train_test_split(test_size=0.2, stratify_by_column="label", seed = global_seed)

In [ ]:
# Pre-process and tokenize data
tokenizer = AutoTokenizer.from_pretrained('prajjwal1/bert-small')

def tokenize(examples):
    preprocessed_texts = [preprocess(text) for text in examples['text']]
    return tokenizer(preprocessed_texts, padding='max_length', truncation=True, max_length = max_length, return_tensors="pt")

tokenized_dataset = labeled_data_for_training.map(tokenize, batched=True)

config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [ ]:
# Import model for linear classification using the number of classes in 1% data.
model = AutoModelForSequenceClassification.from_pretrained('prajjwal1/bert-small', num_labels=num_classes)

pytorch_model.bin:   0%|          | 0.00/116M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Use the 1% labelled data to train a BERT model
latent_num_epochs = 20

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=latent_num_epochs,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_dataset['train'], eval_dataset=tokenized_dataset['test'])

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.344901
2,1.446800,1.343070
3,1.446800,1.339622
4,1.384400,1.334851
5,1.384500,1.328261
6,1.384500,1.320973
7,1.325400,1.312628
8,1.325400,1.303411
9,1.263200,1.291976
10,1.241700,1.282373


TrainOutput(global_step=120, training_loss=1.1516600688298544, metrics={'train_runtime': 4.1347, 'train_samples_per_second': 116.091, 'train_steps_per_second': 29.023, 'total_flos': 9492677591040.0, 'train_loss': 1.1516600688298544, 'epoch': 20.0})

In [ ]:
# Save the model to a directory for local use
model.save_pretrained('./model')
tokenizer.save_pretrained('./tokenizer')

('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/vocab.txt',
 './tokenizer/added_tokens.json',
 './tokenizer/tokenizer.json')

In [ ]:
# Use the model to predict latent classes for rest of training data
def predict_batch(batch):
    # Tokenize the examples
    preprocessed_texts = [preprocess(text) for text in batch['text']]
    inputs = tokenizer(preprocessed_texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Perform the prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted class indices
    predictions = outputs.logits.argmax(-1).cpu().numpy()
    return {'predictions': predictions}

# Create a DataLoader for the rest of the data
rest_of_data_loader = DataLoader(rest_of_data, batch_size=32)

# Run predictions in batches
results = []
for batch in tqdm(rest_of_data_loader, desc="Predicting latent classes"):
    batch_results = predict_batch(batch)
    results.extend(batch_results['predictions'])

Predicting latent classes:   0%|          | 0/93 [00:00<?, ?it/s]

In [ ]:
# Save the predicted classes for local use
file_name = './latent_classes.csv'

with open(file_name, 'w', newline='') as file:
    writer = csv.writer(file)

    for integer in results:
        writer.writerow([integer])

In [ ]:
# Add the latent class to the dataset
def add_new_column(example, idx, new_data):
    example['latent_class'] = new_data[idx]
    return example

labeled_rest_of_data = rest_of_data.map(
    lambda example, idx: add_new_column(example, idx, results),
    with_indices=True
)

# For the labelled data, we use the label as the latent class as it is already known.
def insert_column_with_same_value(example):
    example['latent_class'] = example['label']
    return example

labeled_data = labeled_data.map(
    lambda example: insert_column_with_same_value(example)
)

Map:   0%|          | 0/2970 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [ ]:
# How well does the latent classification work?
def check_columns_equality(example, column1, column2):
    return example[column1] == example[column2]

check_accuracy_dataset = labeled_rest_of_data.filter(lambda example: check_columns_equality(example, 'label', 'latent_class'))

matching_count = len(check_accuracy_dataset)
print(f"Accuracy of latent classification: {matching_count / len(labeled_rest_of_data)}")

Filter:   0%|          | 0/2970 [00:00<?, ? examples/s]

Accuracy of latent classification: 0.7023569023569024


In [ ]:
# Re-merge the 1% data and rest of the data for self-supervised learning
latent_class_data = concatenate_datasets([labeled_data, labeled_rest_of_data])
print(latent_class_data)

Dataset({
    features: ['text', 'label', 'augment_1', 'augment_2', 'translation', 'gpt_1', 'gpt_2', 'latent_class'],
    num_rows: 3000
})


In [ ]:
del results
del model
del tokenizer
del training_args
del trainer
del rest_of_data
del rest_of_data_loader
del labeled_rest_of_data
del check_accuracy_dataset
del labeled_data
del gpt_dataset

# Collect garbage
gc.collect()

108

# Subset selection

Now that we have some form of latent classes assigned to all of the training data, we can find subsets using three methods:

1. Original SAS algorithm
2. Random subset from each latent class
3. Original SAS algorithm + Spanish embeddings

We will select a 20% subset and compare the three. In the long run, we also need to compare with using the whole dataset but that is going to be very computationally intensive.

In [ ]:
# Split into latent classes to select subsets from each
def create_filter_function(latent_class):
    def filter_label(example):
        return example['latent_class'] == latent_class
    return filter_label

world_train_data = latent_class_data.filter(create_filter_function(0))
sports_train_data = latent_class_data.filter(create_filter_function(1))
business_train_data = latent_class_data.filter(create_filter_function(2))
tech_train_data = latent_class_data.filter(create_filter_function(3))

Filter:   0%|          | 0/3000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
# We will select the subset in this ratio for all subsets, for fairness
print(len(world_train_data))
print(len(sports_train_data))
print(len(business_train_data))
print(len(tech_train_data))

703
748
326
1223


## SAS

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('prajjwal1/bert-small')
model = AutoModel.from_pretrained('prajjwal1/bert-small')

def bert_embeddings(data):
    inputs = tokenizer(data, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
    outputs = model(**inputs)
    # Take the first token ([CLS]) embeddings from each sentence
    return outputs.last_hidden_state[:,0,:].detach()

def sas_algorithm(data, subset_size):
    # Get embeddings for the text data
    embeddings = bert_embeddings(data)
    n = len(embeddings)

    # Initialize the subset and similarities
    S = set()
    S_similarities = np.full(n, -np.inf)

    # Compute similarity matrix using cosine_similarity
    similarity_matrix = cosine_similarity(embeddings)

    del embeddings
    gc.collect()

    for _ in tqdm(range(subset_size)):
        not_in_S = np.array([i for i in range(n) if i not in S])

        # Calculate gains for each potential exemplar not in S
        gains = []
        for idx in not_in_S:
            gain = np.sum(np.maximum(S_similarities, similarity_matrix[idx]))
            gains.append(gain)

        # Find the exemplar with the best gain
        best_exemplar_idx = np.argmax(gains)
        best_exemplar = not_in_S[best_exemplar_idx]

        # Update the similarity for the best-selected subset
        S_similarities = np.maximum(S_similarities, similarity_matrix[best_exemplar])
        S.add(best_exemplar)

    return list(S)

In [ ]:
# Run the SAS algorithm to select a 20% subset from each latent class
subset_size = 0.8

world_sas_indices = sas_algorithm(world_train_data['text'], int(subset_size * len(world_train_data)))
sports_sas_indices = sas_algorithm(sports_train_data['text'], int(subset_size * len(sports_train_data)))
business_sas_indices = sas_algorithm(business_train_data['text'], int(subset_size * len(business_train_data)))
tech_sas_indices = sas_algorithm(tech_train_data['text'], int(subset_size * len(tech_train_data)))

  0%|          | 0/562 [00:00<?, ?it/s]

  0%|          | 0/598 [00:00<?, ?it/s]

  0%|          | 0/260 [00:00<?, ?it/s]

  0%|          | 0/978 [00:00<?, ?it/s]

In [ ]:
# Sanity check on indices
print(world_sas_indices)
print(sports_sas_indices)
print(business_sas_indices)
print(tech_sas_indices)

[0, 1, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 16, 18, 20, 21, 25, 26, 27, 28, 29, 30, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 45, 47, 52, 53, 54, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 75, 76, 77, 79, 80, 81, 84, 85, 87, 89, 90, 92, 93, 95, 98, 99, 100, 101, 103, 104, 105, 106, 107, 108, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 131, 132, 133, 134, 135, 136, 138, 139, 140, 141, 142, 143, 144, 146, 147, 148, 149, 151, 152, 153, 154, 155, 156, 157, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 186, 188, 189, 190, 192, 194, 195, 196, 197, 198, 199, 200, 201, 203, 204, 205, 207, 208, 209, 210, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 237, 238, 239, 240, 241, 242, 244, 246, 249, 250, 251, 253, 254, 255, 256, 257, 258, 259, 260, 261, 263, 265, 266, 267, 268, 269, 270, 272, 273, 

In [ ]:
world_sas_subset = world_train_data.select(world_sas_indices)
sports_sas_subset = sports_train_data.select(sports_sas_indices)
business_sas_subset = business_train_data.select(business_sas_indices)
tech_sas_subset = tech_train_data.select(tech_sas_indices)

# Create the subset to use for contrastive self-supervised learning
sas_subset_data = concatenate_datasets([world_sas_subset, sports_sas_subset, business_sas_subset, tech_sas_subset])

In [ ]:
del world_sas_subset
del sports_sas_subset
del business_sas_subset
del tech_sas_subset
del world_sas_indices
del sports_sas_indices
del business_sas_indices
del tech_sas_indices
del tokenizer
del model

gc.collect()

33

In [ ]:
print(sas_subset_data)

Dataset({
    features: ['text', 'label', 'augment_1', 'augment_2', 'translation', 'gpt_1', 'gpt_2', 'latent_class'],
    num_rows: 2398
})


## Random

In [ ]:
# Randomly sample 20% of the dataset for each latent class
def random_subset(data, subset_size):
    return random.sample(range(len(data)), subset_size)

In [ ]:
subset_size = 0.8

world_random_indices = random_subset(world_train_data['text'], int(subset_size * len(world_train_data)))
sports_random_indices = random_subset(sports_train_data['text'], int(subset_size * len(sports_train_data)))
business_random_indices = random_subset(business_train_data['text'], int(subset_size * len(business_train_data)))
tech_random_indices = random_subset(tech_train_data['text'], int(subset_size * len(tech_train_data)))

In [ ]:
# Sanity check on indices
print(world_random_indices)
print(sports_random_indices)
print(business_random_indices)
print(tech_random_indices)

[654, 114, 25, 281, 250, 228, 142, 104, 692, 558, 89, 604, 432, 32, 30, 95, 223, 238, 517, 616, 27, 574, 203, 665, 693, 429, 225, 459, 603, 284, 6, 163, 690, 348, 673, 159, 220, 344, 695, 94, 389, 99, 367, 352, 618, 270, 44, 470, 549, 127, 387, 80, 565, 300, 643, 633, 370, 591, 196, 71, 46, 233, 296, 81, 685, 103, 662, 668, 464, 373, 166, 379, 363, 214, 273, 73, 623, 175, 546, 698, 167, 473, 388, 276, 570, 224, 332, 57, 234, 689, 323, 410, 274, 67, 216, 580, 322, 217, 511, 405, 469, 146, 271, 696, 252, 681, 551, 269, 438, 408, 646, 617, 141, 521, 505, 93, 48, 112, 156, 671, 670, 65, 394, 390, 479, 541, 257, 566, 11, 117, 702, 628, 669, 701, 649, 445, 161, 634, 3, 595, 512, 182, 519, 108, 305, 560, 680, 584, 382, 165, 552, 543, 0, 331, 500, 19, 569, 371, 314, 245, 59, 246, 651, 87, 497, 70, 128, 131, 486, 169, 600, 433, 608, 205, 319, 593, 554, 448, 462, 123, 253, 230, 581, 346, 21, 235, 676, 7, 72, 60, 614, 69, 463, 16, 440, 533, 36, 263, 121, 599, 342, 248, 109, 619, 609, 592, 478, 45

In [ ]:
world_random_subset = world_train_data.select(world_random_indices)
sports_random_subset = sports_train_data.select(sports_random_indices)
business_random_subset = business_train_data.select(business_random_indices)
tech_random_subset = tech_train_data.select(tech_random_indices)

# Create the subset to use for contrastive self-supervised learning
random_subset_data = concatenate_datasets([world_random_subset, sports_random_subset, business_random_subset, tech_random_subset])

In [ ]:
del world_random_subset
del sports_random_subset
del business_random_subset
del tech_random_subset
del world_random_indices
del sports_random_indices
del business_random_indices
del tech_random_indices

gc.collect()

20

In [ ]:
print(random_subset_data)

Dataset({
    features: ['text', 'label', 'augment_1', 'augment_2', 'translation', 'gpt_1', 'gpt_2', 'latent_class'],
    num_rows: 2398
})


## Multi-lingual SAS

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('prajjwal1/bert-small')
model = AutoModel.from_pretrained('prajjwal1/bert-small')

spanish_tokenizer = AutoTokenizer.from_pretrained('dccuchile/albert-tiny-spanish')
spanish_model = AutoModel.from_pretrained('dccuchile/albert-tiny-spanish')

def bert_embeddings(data):
    inputs = tokenizer(data, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
    outputs = model(**inputs)
    # Take the first token ([CLS]) embeddings from each sentence
    return outputs.last_hidden_state[:,0,:].detach()

def albert_embeddings(data):
    inputs = spanish_tokenizer(data, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
    outputs = spanish_model(**inputs)
    # Take the first token ([CLS]) embeddings from each sentence
    return outputs.last_hidden_state[:,0,:].detach()

def sas_with_spanish_algorithm(data, subset_size):
    # Get embeddings for the text data
    embeddings = bert_embeddings(data['text'])
    n = len(embeddings)
    print(embeddings.shape)

    # Initialize the subset and similarities
    S = set()
    S_similarities = np.full(n, -np.inf)

    # Compute similarity matrix using cosine_similarity
    similarity_matrix = cosine_similarity(embeddings)

    del(embeddings)
    gc.collect()

    # Get Spanish embeddings
    spanish_embeddings = albert_embeddings(data['translation'])
    print(spanish_embeddings.shape)

    # Compute similarity matrix using cosine_similarity
    similarity_matrix = np.add(similarity_matrix, cosine_similarity(spanish_embeddings))

    del(spanish_embeddings)
    gc.collect()

    for _ in tqdm(range(subset_size)):
        not_in_S = np.array([i for i in range(n) if i not in S])

        # Calculate gains for each potential exemplar not in S
        gains = []
        for idx in not_in_S:
            gain = np.sum(np.maximum(S_similarities, similarity_matrix[idx]))
            gains.append(gain)

        # Find the exemplar with the best gain
        best_exemplar_idx = np.argmax(gains)
        best_exemplar = not_in_S[best_exemplar_idx]

        # Update the similarity for the best-selected subset
        S_similarities = np.maximum(S_similarities, similarity_matrix[best_exemplar])
        S.add(best_exemplar)

    return list(S)

tokenizer_config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/828 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/21.7M [00:00<?, ?B/s]

In [ ]:
# Run the SAS algorithm to select a 20% subset from each latent class
subset_size = 0.8

world_multilingual_sas_indices = sas_with_spanish_algorithm(world_train_data, int(subset_size * len(world_train_data)))
sports_multilingual_sas_indices = sas_with_spanish_algorithm(sports_train_data, int(subset_size * len(sports_train_data)))
business_multilingual_sas_indices = sas_with_spanish_algorithm(business_train_data, int(subset_size * len(business_train_data)))
tech_multilingual_sas_indices = sas_with_spanish_algorithm(tech_train_data, int(subset_size * len(tech_train_data)))

torch.Size([703, 512])
torch.Size([703, 312])


  0%|          | 0/562 [00:00<?, ?it/s]

torch.Size([748, 512])
torch.Size([748, 312])


  0%|          | 0/598 [00:00<?, ?it/s]

torch.Size([326, 512])
torch.Size([326, 312])


  0%|          | 0/260 [00:00<?, ?it/s]

torch.Size([1223, 512])
torch.Size([1223, 312])


  0%|          | 0/978 [00:00<?, ?it/s]

In [ ]:
world_multilingual_sas_subset = world_train_data.select(world_multilingual_sas_indices)
sports_multilingual_sas_subset = sports_train_data.select(sports_multilingual_sas_indices)
business_multilingual_sas_subset = business_train_data.select(business_multilingual_sas_indices)
tech_multilingual_sas_subset = tech_train_data.select(tech_multilingual_sas_indices)

# Create the subset to use for contrastive self-supervised learning
multilingual_sas_subset_data = concatenate_datasets([world_multilingual_sas_subset, sports_multilingual_sas_subset, business_multilingual_sas_subset, tech_multilingual_sas_subset])

In [ ]:
del world_multilingual_sas_subset
del sports_multilingual_sas_subset
del business_multilingual_sas_subset
del tech_multilingual_sas_subset
del world_multilingual_sas_indices
del sports_multilingual_sas_indices
del business_multilingual_sas_indices
del tech_multilingual_sas_indices
del tokenizer
del model
del spanish_tokenizer
del spanish_model

gc.collect()

33

In [ ]:
print(multilingual_sas_subset_data)

Dataset({
    features: ['text', 'label', 'augment_1', 'augment_2', 'translation', 'gpt_1', 'gpt_2', 'latent_class'],
    num_rows: 2398
})


# Contrastive learning pipeline

Now that we have a 20% representative subset of the data in many different ways. We use this subset, along with two different forms of textual augmentation:

1. Easy data augmentation such as synonym replacement, random addition and random swapping
2. GPT for paraphrasing sentences

to train a self-supervised contrastive learning encoder, which will be evaluated in a downstream prediction task.

In [ ]:
# Define the encoder model. We will use pre-trained BERT as the initial embedding
# and use contrastive learning to further train embeddings that can be tested
# downstream, specifically in news article domain.
model_name = 'prajjwal1/bert-small'

class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def forward(self, text):
        inputs = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = self.model(**inputs)
        return outputs.last_hidden_state[:, 0, :] # Use the [CLS] token as the embedding

# Define the contrastive (InfoNCE) loss
class ContrastiveLoss(nn.Module):
    def forward(self, z_i, z_j, z_k):
        # We use cosine similarities between the embeddings
        cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

        # Similarity with positive augmented examples vs. negative examples
        positive_similarity = torch.exp(cos(z_i, z_j))
        negative_similarity = torch.exp(cos(z_i, z_k))

        loss = -torch.log(positive_similarity / (positive_similarity + negative_similarity))

        return loss.mean()

In [ ]:
# Helper function to randomly move around indices to create a list for negative examples
def shuffle_without_duplication(arr):
    np.random.seed(global_seed)
    arr = np.array(arr)
    n = len(arr)
    # Create an array of the same shape filled with the indices
    indices = np.arange(n)
    while True:
        # Shuffle the indices
        np.random.shuffle(indices)
        # Check if no element remains in its original position
        if not np.any(indices == np.arange(n)):
            break
    # Return the shuffled array
    return arr[indices].tolist()

In [ ]:
# Training function for self-supervised contrastive learning
def self_supervised_training(data, num_epochs, augment='eda'):
    # Parameters
    batch_size = 32
    learning_rate = 1e-6

    # Shuffle the data for better training. At first, all subsets are concatenated and
    # therefore separated by latent class
    data = data.shuffle(seed = global_seed)
    texts = data['text']
    if (augment == 'eda'):
        existing_augments_1 = data['augment_1']
        existing_augments_2 = data['augment_2']
    else:
        existing_augments_1 = data['gpt_1']
        existing_augments_2 = data['gpt_2']

    encoder = Encoder().to(device)
    loss_fn = ContrastiveLoss()
    optimizer = optim.Adam(encoder.parameters(), lr = learning_rate)

    # Training loop
    encoder.train()
    for epoch in range(num_epochs):
        for i in range(0, len(texts), batch_size):
            # Sample a batch of texts
            batch_existing_augments_1 = existing_augments_1[i:i + batch_size]
            batch_existing_augments_2 = existing_augments_2[i:i + batch_size]

            # Clean texts and create a list of negative texts from the batch, ensuring
            # that the same example is not selected for the negative.
            clean_existing_augments_1 = [preprocess(text) for text in batch_existing_augments_1]
            clean_existing_augments_2 = [preprocess(text) for text in batch_existing_augments_2]
            negative_augments = shuffle_without_duplication(clean_existing_augments_1)

            # Get the embeddings from the encoder
            z_i = encoder(clean_existing_augments_1)
            z_j = encoder(clean_existing_augments_2)
            z_k = encoder(negative_augments)

            # Compute the contrastive loss
            loss = loss_fn(z_i, z_j, z_k)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Print every 20 steps
            if (i // batch_size) % 20 == 0:
                print(f'Epoch {epoch}, Step {i // batch_size}, Loss: {loss.item()}')

    return encoder

# Evaluation pipeline

Through contrastive learning, we have learned an encoder $f$ that can embed our news article text for better downstream performance. We test by using this encoder $f$ and the true labels in training a linear classifier head, and testing on the test dataset. Here, we are trying to compare the efficacy of the encoder $f$ for the three different ways of selecting a subset. Additionally, we need to compare with using the full data, which will be left as a future direction due to computational issues.

In [ ]:
# Define the linear classifier head
class LinearClassifier(nn.Module):
    def __init__(self, hidden_size, num_classes):
        super().__init__()
        self.linear = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        return self.linear(x)

In [ ]:
# Create dataloaders from the train and test data
def create_dataloaders(train_data, test_data, batch_size):
    train_val_data = train_data.train_test_split(test_size=0.2, stratify_by_column="label", seed = global_seed)

    train_texts = [preprocess(text) for text in train_val_data['train']['text']]
    val_texts = [preprocess(text) for text in train_val_data['test']['text']]
    test_texts = [preprocess(text) for text in test_data['text']]

    # Create tensors for labels
    train_labels = torch.tensor(train_val_data['train']['label'])
    val_labels = torch.tensor(train_val_data['test']['label'])
    test_labels = torch.tensor(test_data['label'])

    # Create a DataLoader for our training and testing data
    train_data_for_classification = list(zip(train_texts, train_labels))
    val_data_for_classification = list(zip(val_texts, val_labels))
    test_data_for_classification = list(zip(test_texts, test_labels))

    train_dataloader = DataLoader(train_data_for_classification, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_data_for_classification, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_for_classification, batch_size=batch_size)

    return train_dataloader, val_dataloader, test_dataloader

# Evaluation function to train a linear classifier head on top of the learned encoder
# and evaluate on the test set
def evaluate_encoder(encoder, train_data, test_data):
    # Parameters
    batch_size = 64
    num_epochs = 5
    num_classes = 4
    learning_rate = 1e-4

    # Create dataloaders
    train_dataloader, val_dataloader, test_dataloader = create_dataloaders(train_data, test_data, batch_size)

    # Set the encoder to evaluation mode and freeze all layers to test the trained embeddings from contrastive learning
    encoder.eval()
    for param in encoder.parameters():
        param.requires_grad = False

    # Initialize the linear classifier head
    classifier = LinearClassifier(encoder.model.config.hidden_size, num_classes).to(device)

    # Loss function and optimizer for the classifier head
    # Define scheduler for learning rate.
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(), lr = learning_rate)

    best_val_loss = float('inf')
    checkpoint_path = './best_checkpoint.pth'

    # Train the classifier
    for epoch in range(num_epochs):
        classifier.train()
        train_loss = 0.0

        for (index, data) in enumerate(train_dataloader):
            texts, labels = data
            labels = labels.to(device)
            optimizer.zero_grad()

            # Forward pass through the frozen encoder and classifier head
            embeddings = encoder(texts)
            outputs = classifier(embeddings)
            loss = criterion(outputs, labels)

            # Backpropagation
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            # Print every 20 steps
            if index % 50 == 0:
                print(f'Epoch {epoch}, Step {index}, Loss: {loss.item()}')

        classifier.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data in val_dataloader:
                texts, labels = data
                labels = labels.to(device)

                embeddings = encoder(texts)
                outputs = classifier(embeddings)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        # Average losses
        train_loss /= len(train_dataloader.dataset)
        val_loss /= len(val_dataloader.dataset)

        # Print stats
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

        # Save the best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(classifier.state_dict(), checkpoint_path)

    # Evaluate the classifier
    classifier.load_state_dict(torch.load(checkpoint_path))
    classifier.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for texts, labels in test_dataloader:
            labels = labels.to(device)

            embeddings = encoder(texts)
            outputs = classifier(embeddings)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    print(f'Accuracy on the test set: {accuracy:.4f}')

# Testing different subset selection processes + augmentation techniques

## Subset selection using SAS

### GPT

In [ ]:
encoder = self_supervised_training(sas_subset_data, 100, 'gpt')

Epoch 0, Step 0, Loss: 0.5657941102981567
Epoch 0, Step 20, Loss: 0.5520174503326416
Epoch 0, Step 40, Loss: 0.5499248504638672
Epoch 0, Step 60, Loss: 0.5359369516372681
Epoch 1, Step 0, Loss: 0.5098947286605835
Epoch 1, Step 20, Loss: 0.5009803175926208
Epoch 1, Step 40, Loss: 0.5093869566917419
Epoch 1, Step 60, Loss: 0.5032669305801392
Epoch 2, Step 0, Loss: 0.4675427973270416
Epoch 2, Step 20, Loss: 0.47516509890556335
Epoch 2, Step 40, Loss: 0.47859305143356323
Epoch 2, Step 60, Loss: 0.47941160202026367
Epoch 3, Step 0, Loss: 0.43944352865219116
Epoch 3, Step 20, Loss: 0.45015597343444824
Epoch 3, Step 40, Loss: 0.4602309465408325
Epoch 3, Step 60, Loss: 0.459994912147522
Epoch 4, Step 0, Loss: 0.4231398105621338
Epoch 4, Step 20, Loss: 0.439696729183197
Epoch 4, Step 40, Loss: 0.44875699281692505
Epoch 4, Step 60, Loss: 0.4542897939682007
Epoch 5, Step 0, Loss: 0.41239744424819946
Epoch 5, Step 20, Loss: 0.4282247722148895
Epoch 5, Step 40, Loss: 0.4355611503124237
Epoch 5, Ste

In [ ]:
evaluate_encoder(encoder, train_data, test_data)

Epoch 0, Step 0, Loss: 1.0662610530853271
Epoch 1, Train Loss: 0.0142, Val Loss: 0.0121
Epoch 1, Step 0, Loss: 0.8205508589744568
Epoch 2, Train Loss: 0.0106, Val Loss: 0.0098
Epoch 2, Step 0, Loss: 0.5516529083251953
Epoch 3, Train Loss: 0.0092, Val Loss: 0.0087
Epoch 3, Step 0, Loss: 0.48459336161613464
Epoch 4, Train Loss: 0.0085, Val Loss: 0.0082
Epoch 4, Step 0, Loss: 0.541629433631897
Epoch 5, Train Loss: 0.0081, Val Loss: 0.0079
Accuracy on the test set: 0.8254


### EDA

In [ ]:
encoder = self_supervised_training(sas_subset_data, 100, 'eda')

Epoch 0, Step 0, Loss: 0.5617918968200684
Epoch 0, Step 20, Loss: 0.5525816679000854
Epoch 0, Step 40, Loss: 0.546532154083252
Epoch 0, Step 60, Loss: 0.5376178026199341
Epoch 1, Step 0, Loss: 0.5168556571006775
Epoch 1, Step 20, Loss: 0.5117779970169067
Epoch 1, Step 40, Loss: 0.5084371566772461
Epoch 1, Step 60, Loss: 0.49710893630981445
Epoch 2, Step 0, Loss: 0.48188939690589905
Epoch 2, Step 20, Loss: 0.47592008113861084
Epoch 2, Step 40, Loss: 0.4772654175758362
Epoch 2, Step 60, Loss: 0.47386544942855835
Epoch 3, Step 0, Loss: 0.4491831958293915
Epoch 3, Step 20, Loss: 0.4488740563392639
Epoch 3, Step 40, Loss: 0.4594321846961975
Epoch 3, Step 60, Loss: 0.4559534788131714
Epoch 4, Step 0, Loss: 0.4350602626800537
Epoch 4, Step 20, Loss: 0.4373979866504669
Epoch 4, Step 40, Loss: 0.4412659704685211
Epoch 4, Step 60, Loss: 0.4508996903896332
Epoch 5, Step 0, Loss: 0.41770273447036743
Epoch 5, Step 20, Loss: 0.4302414059638977
Epoch 5, Step 40, Loss: 0.4286769926548004
Epoch 5, Step

In [ ]:
evaluate_encoder(encoder, train_data, test_data)

Epoch 0, Step 0, Loss: 1.650473952293396
Epoch 1, Train Loss: 0.0210, Val Loss: 0.0173
Epoch 1, Step 0, Loss: 1.0161110162734985
Epoch 2, Train Loss: 0.0144, Val Loss: 0.0129
Epoch 2, Step 0, Loss: 0.791535496711731
Epoch 3, Train Loss: 0.0116, Val Loss: 0.0110
Epoch 3, Step 0, Loss: 0.6018061637878418
Epoch 4, Train Loss: 0.0104, Val Loss: 0.0100
Epoch 4, Step 0, Loss: 0.6115778684616089
Epoch 5, Train Loss: 0.0097, Val Loss: 0.0094
Accuracy on the test set: 0.8037


## Random subset selection

### GPT

In [ ]:
encoder = self_supervised_training(random_subset_data, 100, 'gpt')

Epoch 0, Step 0, Loss: 0.568693220615387
Epoch 0, Step 20, Loss: 0.5380946397781372
Epoch 0, Step 40, Loss: 0.5268967151641846
Epoch 0, Step 60, Loss: 0.5169251561164856
Epoch 1, Step 0, Loss: 0.5218536853790283
Epoch 1, Step 20, Loss: 0.4921281933784485
Epoch 1, Step 40, Loss: 0.4823276400566101
Epoch 1, Step 60, Loss: 0.48105135560035706
Epoch 2, Step 0, Loss: 0.4872516393661499
Epoch 2, Step 20, Loss: 0.46868905425071716
Epoch 2, Step 40, Loss: 0.4523528814315796
Epoch 2, Step 60, Loss: 0.46070531010627747
Epoch 3, Step 0, Loss: 0.4630383253097534
Epoch 3, Step 20, Loss: 0.450221985578537
Epoch 3, Step 40, Loss: 0.4309575855731964
Epoch 3, Step 60, Loss: 0.4418901205062866
Epoch 4, Step 0, Loss: 0.44244080781936646
Epoch 4, Step 20, Loss: 0.44221973419189453
Epoch 4, Step 40, Loss: 0.4227319657802582
Epoch 4, Step 60, Loss: 0.4385504722595215
Epoch 5, Step 0, Loss: 0.43701428174972534
Epoch 5, Step 20, Loss: 0.4281504154205322
Epoch 5, Step 40, Loss: 0.40842360258102417
Epoch 5, Ste

In [ ]:
evaluate_encoder(encoder, train_data, test_data)

Epoch 0, Step 0, Loss: 1.3772385120391846
Epoch 1, Train Loss: 0.0190, Val Loss: 0.0163
Epoch 1, Step 0, Loss: 0.9740433096885681
Epoch 2, Train Loss: 0.0136, Val Loss: 0.0125
Epoch 2, Step 0, Loss: 0.702680230140686
Epoch 3, Train Loss: 0.0110, Val Loss: 0.0107
Epoch 3, Step 0, Loss: 0.5621753931045532
Epoch 4, Train Loss: 0.0096, Val Loss: 0.0097
Epoch 4, Step 0, Loss: 0.43274688720703125
Epoch 5, Train Loss: 0.0089, Val Loss: 0.0091
Accuracy on the test set: 0.8249


### EDA

In [ ]:
encoder = self_supervised_training(random_subset_data, 100, 'eda')

Epoch 0, Step 0, Loss: 0.5665410161018372
Epoch 0, Step 20, Loss: 0.5504021644592285
Epoch 0, Step 40, Loss: 0.539528489112854
Epoch 0, Step 60, Loss: 0.532413125038147
Epoch 1, Step 0, Loss: 0.52592533826828
Epoch 1, Step 20, Loss: 0.512860894203186
Epoch 1, Step 40, Loss: 0.4987344741821289
Epoch 1, Step 60, Loss: 0.4895690679550171
Epoch 2, Step 0, Loss: 0.48677265644073486
Epoch 2, Step 20, Loss: 0.48806875944137573
Epoch 2, Step 40, Loss: 0.4621196389198303
Epoch 2, Step 60, Loss: 0.4713185429573059
Epoch 3, Step 0, Loss: 0.4625779390335083
Epoch 3, Step 20, Loss: 0.4691559970378876
Epoch 3, Step 40, Loss: 0.44805073738098145
Epoch 3, Step 60, Loss: 0.4526069164276123
Epoch 4, Step 0, Loss: 0.44508910179138184
Epoch 4, Step 20, Loss: 0.4527443051338196
Epoch 4, Step 40, Loss: 0.43156683444976807
Epoch 4, Step 60, Loss: 0.4391680359840393
Epoch 5, Step 0, Loss: 0.4322054982185364
Epoch 5, Step 20, Loss: 0.4438040852546692
Epoch 5, Step 40, Loss: 0.42057961225509644
Epoch 5, Step 60

In [ ]:
evaluate_encoder(encoder, train_data, test_data)

Epoch 0, Step 0, Loss: 1.45023775100708
Epoch 1, Train Loss: 0.0195, Val Loss: 0.0165
Epoch 1, Step 0, Loss: 0.9981200098991394
Epoch 2, Train Loss: 0.0139, Val Loss: 0.0126
Epoch 2, Step 0, Loss: 0.750274658203125
Epoch 3, Train Loss: 0.0113, Val Loss: 0.0108
Epoch 3, Step 0, Loss: 0.5729748606681824
Epoch 4, Train Loss: 0.0099, Val Loss: 0.0098
Epoch 4, Step 0, Loss: 0.5968368053436279
Epoch 5, Train Loss: 0.0092, Val Loss: 0.0093
Accuracy on the test set: 0.8099


## Subset selection using multilingual SAS

### GPT

In [ ]:
encoder = self_supervised_training(multilingual_sas_subset_data, 100, 'gpt')

Epoch 0, Step 0, Loss: 0.5637630224227905
Epoch 0, Step 20, Loss: 0.5534616708755493
Epoch 0, Step 40, Loss: 0.5337893962860107
Epoch 0, Step 60, Loss: 0.5319439172744751
Epoch 1, Step 0, Loss: 0.5132808089256287
Epoch 1, Step 20, Loss: 0.5029915571212769
Epoch 1, Step 40, Loss: 0.4947000741958618
Epoch 1, Step 60, Loss: 0.49288028478622437
Epoch 2, Step 0, Loss: 0.4742392897605896
Epoch 2, Step 20, Loss: 0.47302740812301636
Epoch 2, Step 40, Loss: 0.46954452991485596
Epoch 2, Step 60, Loss: 0.4686645567417145
Epoch 3, Step 0, Loss: 0.4495818316936493
Epoch 3, Step 20, Loss: 0.4468485713005066
Epoch 3, Step 40, Loss: 0.4508761465549469
Epoch 3, Step 60, Loss: 0.45269525051116943
Epoch 4, Step 0, Loss: 0.43570834398269653
Epoch 4, Step 20, Loss: 0.4371008276939392
Epoch 4, Step 40, Loss: 0.4385424852371216
Epoch 4, Step 60, Loss: 0.44372326135635376
Epoch 5, Step 0, Loss: 0.42270350456237793
Epoch 5, Step 20, Loss: 0.4260256588459015
Epoch 5, Step 40, Loss: 0.43045052886009216
Epoch 5, 

In [ ]:
evaluate_encoder(encoder, train_data, test_data)

Epoch 0, Step 0, Loss: 1.5340090990066528
Epoch 1, Train Loss: 0.0202, Val Loss: 0.0171
Epoch 1, Step 0, Loss: 1.0458886623382568
Epoch 2, Train Loss: 0.0140, Val Loss: 0.0129
Epoch 2, Step 0, Loss: 0.9111129641532898
Epoch 3, Train Loss: 0.0112, Val Loss: 0.0110
Epoch 3, Step 0, Loss: 0.5029406547546387
Epoch 4, Train Loss: 0.0099, Val Loss: 0.0100
Epoch 4, Step 0, Loss: 0.5765625834465027
Epoch 5, Train Loss: 0.0092, Val Loss: 0.0094
Accuracy on the test set: 0.8020


### EDA

In [ ]:
encoder = self_supervised_training(multilingual_sas_subset_data, 100, 'eda')

Epoch 0, Step 0, Loss: 0.5674014091491699
Epoch 0, Step 20, Loss: 0.5566194653511047
Epoch 0, Step 40, Loss: 0.551145613193512
Epoch 0, Step 60, Loss: 0.5446450710296631
Epoch 1, Step 0, Loss: 0.5257402658462524
Epoch 1, Step 20, Loss: 0.5089420080184937
Epoch 1, Step 40, Loss: 0.5104547739028931
Epoch 1, Step 60, Loss: 0.5169914960861206
Epoch 2, Step 0, Loss: 0.4847622811794281
Epoch 2, Step 20, Loss: 0.47765403985977173
Epoch 2, Step 40, Loss: 0.4747188091278076
Epoch 2, Step 60, Loss: 0.48749157786369324
Epoch 3, Step 0, Loss: 0.4548238515853882
Epoch 3, Step 20, Loss: 0.45880427956581116
Epoch 3, Step 40, Loss: 0.4529693126678467
Epoch 3, Step 60, Loss: 0.46723631024360657
Epoch 4, Step 0, Loss: 0.43895915150642395
Epoch 4, Step 20, Loss: 0.44051581621170044
Epoch 4, Step 40, Loss: 0.44205206632614136
Epoch 4, Step 60, Loss: 0.4573732614517212
Epoch 5, Step 0, Loss: 0.4246271848678589
Epoch 5, Step 20, Loss: 0.43375951051712036
Epoch 5, Step 40, Loss: 0.4319568872451782
Epoch 5, S

In [ ]:
evaluate_encoder(encoder, train_data, test_data)

Epoch 0, Step 0, Loss: 1.4415496587753296
Epoch 1, Train Loss: 0.0186, Val Loss: 0.0160
Epoch 1, Step 0, Loss: 0.9554000496864319
Epoch 2, Train Loss: 0.0127, Val Loss: 0.0120
Epoch 2, Step 0, Loss: 0.7540596723556519
Epoch 3, Train Loss: 0.0101, Val Loss: 0.0104
Epoch 3, Step 0, Loss: 0.5645879507064819
Epoch 4, Train Loss: 0.0090, Val Loss: 0.0096
Epoch 4, Step 0, Loss: 0.5694847106933594
Epoch 5, Train Loss: 0.0084, Val Loss: 0.0092
Accuracy on the test set: 0.8097


## Baseline with full data

### GPT

In [ ]:
# encoder = self_supervised_training(train_data, 100, 'gpt')

In [ ]:
# evaluate_encoder(encoder, train_data, test_data)

### EDA

In [ ]:
# encoder = self_supervised_training(train_data, 100, 'eda')

In [ ]:
# evaluate_encoder(encoder, train_data, test_data)

# TODO: Further research to come.